In [4]:
"""
pytorch 流程总结

train_loader=DataLoader
test_loader=DataLoader

# 构建模型 
class ConvNet(nn.Module)

# 初始化模型 
model = ConvNet().to(DEVICE)

# 定义优化器 
optimizer = optim.Adam(model.parameters())

# 切换到训练模式 
model.train()

# 遍历训练数据
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    
    optimizer.zero_grad()        
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

"""

SyntaxError: invalid syntax (<ipython-input-4-5dc7913d422c>, line 1)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.1.0'

In [8]:
BATCH_SIZE=512 #大概需要2G的显存
EPOCHS=20 # 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多

In [9]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [10]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.log_softmax(out,dim=1)
        return out

In [11]:
def train(model, device, train_loader, optimizer, epoch):
    # 切换到训练模式，比如dropout的状态等等。用处不大。
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # 先梯度清零(与net.zero_grad()效果一样)，必须在loss.backward()之前执行。
        optimizer.zero_grad()
        
        output = model(data)
        # 定义损失，定义损失之前，得准备好output
        loss = F.nll_loss(output, target)
        
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step()
        
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    # 切换评估模式
    model.eval()
    
    # 测试损失
    test_loss = 0
    # 正确评估的个数
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            
            # 通过output计算测试损失
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            
            # 通过output统计正确评估的个数
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.341055
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.191121
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.119926

Test set: Average loss: 0.0947, Accuracy: 9732/10000 (97%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.128587
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.090210
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.053235

Test set: Average loss: 0.0578, Accuracy: 9818/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.036562
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.038381
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.061712

Test set: Average loss: 0.0452, Accuracy: 9853/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.049622
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.079077
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.024760

Test set: Average loss: 0.0410, Accuracy: 9865/10000 (99%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.028916
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.029611
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.049095

Test 

In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split

device = 'cuda' if torch.cuda.is_available() else 'cpu'

np.random.seed(42)
x = np.random.rand(100, 1)
true_a, true_b = 1, 2
y = true_a + true_b*x + 0.1*np.random.randn(100, 1)

x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

dataset = TensorDataset(x_tensor, y_tensor) # dataset = CustomDataset(x_tensor, y_tensor)

train_dataset, val_dataset = random_split(dataset, [80, 20])

train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(y, yhat)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()
    return train_step

# Estimate a and b
torch.manual_seed(42)

model = ManualLinearRegression().to(device) # model = nn.Sequential(nn.Linear(1, 1)).to(device)
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train_step = make_train_step(model, loss_fn, optimizer)

n_epochs = 100
training_losses = []
validation_losses = []
print(model.state_dict())

for epoch in range(n_epochs):
    batch_losses = []
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        loss = train_step(x_batch, y_batch)
        batch_losses.append(loss)
    training_loss = np.mean(batch_losses)
    training_losses.append(training_loss)

    with torch.no_grad():
        val_losses = []
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            model.eval()
            yhat = model(x_val)
            val_loss = loss_fn(y_val, yhat).item()
            val_losses.append(val_loss)
        validation_loss = np.mean(val_losses)
        validation_losses.append(validation_loss)

    print(f"[{epoch+1}] Training loss: {training_loss:.3f}\t Validation loss: {validation_loss:.3f}")

print(model.state_dict())